In [4]:
using JuMP, Ipopt

# define JuMP model object
m = Model(solver = IpoptSolver(tol = 1e-6, max_iter = 200, print_level = 0));

# define sets 
W = ["w1","w2","w3"]; # waste streams
T = ["tA","tB"];      # waste treatment facilities

# define data
fin = [10;20;10];           # waste flows (gpm)
fin = Dict(zip(W,fin)); 
Cin = [0.1; 0.5; 0.25];     # waste concentration (kg/gpm)
Cin = Dict(zip(W,Cin)); 
fmax = [20; 30];            # max waste flow capacity (gpm)
fmax = Dict(zip(T,fmax)); 
Cmax = [0.4; 0.3];          # max waste concentration capacity (kg/gpm)
Cmax = Dict(zip(T,Cmax));
α = [50; 10]                # processing cost (usd/gpm)
α = Dict(zip(T,α));         

# define optimization problem
@variables m begin
    f[W,T]>=0     # waste to treatment flow
    ftot[T]>=0    # treatment inlet flow
    Ctot[T]>=0    # treatment inlet concentration
end
@constraint(m,[w in W], fin[w] == sum(f[w,t] for t in T))  
@constraint(m, λ[t in T], ftot[t] == sum(f[w,t] for w in W)) 
@constraint(m,[t in T], Ctot[t]*ftot[t] == sum(f[w,t]*Cin[w] for w in W)) 
@constraint(m, [t in T], Ctot[t] <= Cmax[t])
@constraint(m, [t in T], ftot[t] <= fmax[t])
@objective(m, Min, sum(α[t]*ftot[t] for t in T))
solve(m)

# get solution
println(getvalue(ftot))
println(getvalue(Ctot))
println(getvalue(f))
println(getdual(λ))

ftot: 1 dimensions:
[tA] = 14.999996004541718
[tB] = 25.000003995458282
Ctot: 1 dimensions:
[tA] = 0.4000000099848527
[tB] = 0.30000000999091847
f: 2 dimensions:
[w1,:]
  [w1,tA] = 2.0466180073504843
  [w1,tB] = 7.953381992649517
[w2,:]
  [w2,tA] = 10.227969006227072
  [w2,tB] = 9.772030993772928
[w3,:]
  [w3,tA] = 2.725408990964162
  [w3,tB] = 7.274591009035837
__anon__: 1 dimensions:
[tA] = -110.00000391568517
[tB] = -110.00000395740864
